In [1]:
import optuna
import json
import pprint
from uuid import uuid4

# Third Party Library
import pandas as pd
from ex_utils.config.paths import get_dataset_path
from ex_utils.config.quality_metrics import qm_names
from ex_utils.share import (
    ex_path,
    generate_seed_median_df,
    generate_sscalers,
)
from ex_utils.utils.graph import (
    load_nx_graph,
    nx_graph_preprocessing,
)

# Standard Library
import argparse
from time import perf_counter

# Third Party Library
import matplotlib.pyplot as plt
import networkx as nx
import optuna
import pandas as pd
from egraph import Drawing, all_sources_bfs
from ex_utils.config.dataset import dataset_names
from ex_utils.config.paths import get_dataset_path
from ex_utils.config.quality_metrics import qm_names
from ex_utils.share import (
    draw,
    ex_path,
    generate_base_df_data,
    rate2pivots,
    draw_and_measure,
)
from ex_utils.utils.graph import (
    egraph_graph,
    load_nx_graph,
    nx_graph_preprocessing,
)
from tqdm import tqdm
import math


export_data = {}

EDGE_WEIGHT = 30
d_names = sorted(
    [
        "1138_bus",
        "USpowerGrid",
        "dwt_1005",
        "poli",
        "qh882",
    ]
)

seeds = list(range(15))
n_split = 10
multi_n_trials = 300
single_n_trials = 300

pref_array = [0.5, 0.5, 0.5, 5.0, 2.0, 0.5, 4.0, 0.5, 4.0, 0.5]
pref = dict(zip(qm_names, pref_array))
drawing_seeds = list(range(3))

params = dict(
    zip(
        d_names,
        [
            {"empirical": {"pivots": 50, "iterations": 100, "eps": 0.1}}
            for _ in range(len(d_names))
        ],
    )
)


for d_name in d_names:
    export_data[d_name] = []
    df_paths = [
        ex_path.joinpath(
            f"data/grid/{d_name}/n_split={n_split}/seed={seed}.pkl"
        )
        for seed in seeds
    ]
    df = pd.concat([pd.read_pickle(path) for path in df_paths])
    df = generate_seed_median_df(df)
    scalers = generate_sscalers(df)

    dataset_path = get_dataset_path(d_name)
    nx_graph = nx_graph_preprocessing(
        load_nx_graph(dataset_path=dataset_path), EDGE_WEIGHT
    )
    p_max = max(1, int(len(nx_graph.nodes) * 0.25))
    eg_graph, eg_indices = egraph_graph(nx_graph=nx_graph)
    eg_distance_matrix = all_sources_bfs(eg_graph, EDGE_WEIGHT)

    multi_obj_study_name = f"multi-obj-{multi_n_trials}"
    multi_db_uri = f'sqlite:///{ex_path.joinpath(f"data/optimization/{d_name}-multi-obj.db")}'
    multi_obj_study = optuna.load_study(
        storage=multi_db_uri, study_name=multi_obj_study_name
    )
    pareto_front = list(multi_obj_study.best_trials)

    # multi
    multi = -float("inf")
    for pareto_optimal in pareto_front:
        pareto_optimal_quality_metrics = pareto_optimal.user_attrs[
            "median_quality_metrics"
        ]
        pareto_optimal_scaled_quality_metrics = dict(
            [
                (
                    qm_name,
                    scalers[qm_name].transform(
                        [[pareto_optimal_quality_metrics[qm_name]]]
                    )[0][0],
                )
                for qm_name in qm_names
            ]
        )
        pareto_optimal_scaled_quality_metrics_sum = sum(
            [
                pareto_optimal_scaled_quality_metrics[qm_name] * pref[qm_name]
                for qm_name in qm_names
            ]
        )

        if multi < pareto_optimal_scaled_quality_metrics_sum:
            multi = pareto_optimal_scaled_quality_metrics_sum
            params[d_name]["multi"] = pareto_optimal.params

    # single
    single_obj_study_name = f'single-obj_{",".join(map(str, pref_array))}'
    single_db_uri = f'sqlite:///{ex_path.joinpath(f"data/optimization/{d_name}-user-experiment.db")}'
    sinigle_obj_study = optuna.load_study(
        storage=single_db_uri, study_name=single_obj_study_name
    )

    params[d_name]["single"] = sinigle_obj_study.best_trial.params
    sigle = sinigle_obj_study.best_trial.value

    empirical = dict(zip(qm_names, [0 for _ in len(qm_names)]))
    multi = dict(zip(qm_names, [0 for _ in len(qm_names)]))
    single = dict(zip(qm_names, [0 for _ in len(qm_names)]))

    # for d_seed in drawing_seeds:
    #     eg_drawing = Drawing.initial_placement(eg_graph)
    #     pos, quality_metrics, em_scaled_quality_metrics = draw_and_measure(
    #         pivots=params["empirical"]["pivots"],
    #         iterations=params["empirical"]["iterations"],
    #         eps=params["empirical"]["eps"],
    #         eg_graph=eg_graph,
    #         eg_indices=eg_indices,
    #         eg_drawing=eg_drawing,
    #         edge_weight=EDGE_WEIGHT,
    #         seed=d_seed,
    #         n_nodes=len(nx_graph.nodes),
    #         n_edges=len(nx_graph.edges),
    #         scalers=scalers,
    #     )

    #     eg_drawing = Drawing.initial_placement(eg_graph)
    #     pos, quality_metrics, mu_scaled_quality_metrics = draw_and_measure(
    #         pivots=params["multi"]["pivots"],
    #         iterations=params["multi"]["iterations"],
    #         eps=params["multi"]["eps"],
    #         eg_graph=eg_graph,
    #         eg_indices=eg_indices,
    #         eg_drawing=eg_drawing,
    #         edge_weight=EDGE_WEIGHT,
    #         seed=d_seed,
    #         n_nodes=len(nx_graph.nodes),
    #         n_edges=len(nx_graph.edges),
    #         scalers=scalers,
    #     )

    #     eg_drawing = Drawing.initial_placement(eg_graph)
    #     pos, quality_metrics, si_scaled_quality_metrics = draw_and_measure(
    #         pivots=params["single"]["pivots"],
    #         iterations=params["single"]["iterations"],
    #         eps=params["single"]["eps"],
    #         eg_graph=eg_graph,
    #         eg_indices=eg_indices,
    #         eg_drawing=eg_drawing,
    #         edge_weight=EDGE_WEIGHT,
    #         seed=d_seed,
    #         n_nodes=len(nx_graph.nodes),
    #         n_edges=len(nx_graph.edges),
    #         scalers=scalers,
    #     )

    #     for qm_name in qm_names:
    #         empirical[qm_name] += em_scaled_quality_metrics[]

/Users/fuga_takata/dev/vdslab-project/hyperparameter-in-graph-drawing/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from random import shuffle

for d_name in d_names:
    shuffle(export_data[d_name])

with open(ex_path.joinpath("data/ex2/data.json"), mode="w") as f:
    json.dump(export_data, f)